Problem Description

Travelling in and around Seatle City is not unlike any situation travelling around any city or on any public raodway.  When traffic stops or backs up for miles, drivers are often wondering how long it will take to clear and if an alternate route should be found.  Modern vehicles with modern GPS systems use Predictive Traffic feature which gives the driver an understanding of of the traffic patterns and possible delays using real time traffic monitoring and integrated navigation to find a more efficinet route in th event of an accident or slowdown.  In this analysis I want to use the shared sample data set to predict the severity of an accident.  This could be used to feed such a system or an electronic highway sign to re-route traffic in the case of a severe accident or fatality.

Data Desciption

I plan to use the share data set example of Seattle City for my analysis.  The data requires balancing, pre-processing and cleaning before it can be used to train and test the model.  The model will use the severity code as the target label and the rest of the labels, as determined by pre-processing and exploratory analysis, will be used as predictors.  The following is a preliminary download and exploration of the data set.

In [2]:
# Importing the necessary tools for the preliminary exploration of the data.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [4]:
# Downloading the sample data set

In [5]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
print('Download Completed!')

--2020-09-14 16:54:29--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  45.6MB/s   in 1.5s   

2020-09-14 16:54:31 (45.6 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]

Download Completed!


In [6]:
# Initial look at the columns and label types

In [7]:
df = pd.read_csv('Data-Collisions.csv')
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [8]:
# determining if the data set has enough rows for analysis

In [9]:
df.shape

(194673, 38)

In [10]:
# Determining missing data

In [11]:
missing_data = df.isnull()
missing_data.head(5)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False


In [12]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64

OBJECTID
False    194673
Name: OBJECTID, dtype: int64

INCKEY
False    194673
Name: INCKEY, dtype: int64

COLDETKEY
False    194673
Name: COLDETKEY, dtype: int64

REPORTNO
False    194673
Name: REPORTNO, dtype: int64

STATUS
False    194673
Name: STATUS, dtype: int64

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

INTKEY
True     129603
False     65070
Name: INTKEY, dtype: int64

LOCATION
False    191996
True       2677
Name: LOCATION, dtype: int64

EXCEPTRSNCODE
True     109862
False     84811
Name: EXCEPTRSNCODE, dtype: int64

EXCEPTRSNDESC
True     189035
False      5638
Name: EXCEPTRSNDESC, dtype: int64

SEVERITYCODE.1
False    194673
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    194673
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    189769
True       4904
Name: C

In [13]:
# Determimng data types for potential mis-matches

In [14]:
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [15]:
# The following block of code is used for iteratively for labels from above to determine the counts of each type

In [16]:
df['ADDRTYPE'].value_counts()

Block           126926
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64

In [17]:
# Code below is to determine balancing requirements of potential labels identified above

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

In [19]:
# Dropping of some of the label determined as non essential to the analysis (some may be added back at a later time)

In [20]:
df.drop(['X','Y','OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','INTKEY','LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC','SEVERITYCODE.1','PEDROWNOTGRNT','INCDATE','SDOTCOLNUM','ST_COLDESC','UNDERINFL','SDOT_COLDESC','INATTENTIONIND','INCDTTM'], axis=1, inplace=True)

In [21]:
# Another look at the data

In [22]:
df.head()

,SEVERITYCODE,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),11,Overcast,Wet,Daylight,NaN,10,0,0,N
1,1,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),16,Raining,Wet,Dark - Street Lights On,NaN,11,0,0,N
2,1,Block,Property Damage Only Collision,Parked Car,4,0,0,3,Mid-Block (not related to intersection),14,Overcast,Dry,Daylight,NaN,32,0,0,N
3,1,Block,Property Damage Only Collision,Other,3,0,0,3,Mid-Block (not related to intersection),11,Clear,Dry,Daylight,NaN,23,0,0,N
4,2,Intersection,Injury Collision,Angles,2,0,0,2,At Intersection (intersection related),11,Raining,Wet,Daylight,NaN,10,0,0,N


In [23]:
# Value counts below are used iteratively as a guide to replacing object type to int. in the code several lines below

In [24]:
test_count=df['JUNCTIONTYPE'].value_counts().to_frame()

In [25]:
test_count

,JUNCTIONTYPE
Mid-Block (not related to intersection),89800
At Intersection (intersection related),62810
Mid-Block (but intersection related),22790
Driveway Junction,10671
At Intersection (but not related to intersection),2098
Ramp Junction,166
Unknown,9


In [26]:
df['ADDRTYPE'] = df['ADDRTYPE'].replace(['Block','Intersection','Alley'],['1','2','3'])

In [27]:
df['SEVERITYDESC'] = df['SEVERITYDESC'].replace(['Property Damage Only Collision','Injury Collision'],['1','2'])

In [28]:
df['COLLISIONTYPE'] = df['COLLISIONTYPE'].replace(['Parked Car','Angles','Rear Ended','Other','Sideswipe','Left Turn','Pedestrian','Cycles','RightTurn','Head On'],['1','2','3','4','5','6','7','8','9','10'])

In [29]:
df['WEATHER'] = df['WEATHER'].replace(['Clear','Raining','Overcast','Unknown','Snowing','Other','Fog/Smog/Smoke','Sleet/Hail/Freezing Rain','Blowing Sand/Dirt','Severe Crosswind','Partly Cloudy'],['1','2','3','4','5','6','7','8','9','10','11'])

In [30]:
df['ROADCOND'] = df['ROADCOND'].replace(['Dry','Wet','Unknown','Ice','Snow/Slush','Other','Standing Water','Sand/Mud dirt','Oil'],['1','2','3','4','5','6','7','8','9'])

In [31]:
df['LIGHTCOND'] = df['LIGHTCOND'].replace(['Daylight','Dark - Street Lights On','Unknown','Dusk','Dawn','Dark - No Street Lights','Dark - Street Lights Off','Other','Dark - Unknown Lighting'],['1','2','3','4','5','6','7','8','9'])

In [32]:
df['SPEEDING'] = df['SPEEDING'].replace(['Y'],['1'])

In [33]:
df['JUNCTIONTYPE'] = df['JUNCTIONTYPE'].replace(['Mid-Block (not related to intersection)','At Intersection (intersection related)','Mid-Block (but intersection related)','Driveway Junction','At Intersection (but not related to intersection)','Ramp Junction','Unknown'],['1','2','3','4','5','6','7'])

In [34]:
df.head()

,SEVERITYCODE,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,2,2,2,2,0,0,2,2,11,3,2,1,NaN,10,0,0,N
1,1,1,1,5,2,0,0,2,1,16,2,2,2,NaN,11,0,0,N
2,1,1,1,1,4,0,0,3,1,14,3,1,1,NaN,32,0,0,N
3,1,1,1,4,3,0,0,3,1,11,1,1,1,NaN,23,0,0,N
4,2,2,2,2,2,0,0,2,2,11,2,2,1,NaN,10,0,0,N


In [35]:
df.head()

,SEVERITYCODE,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,2,2,2,2,0,0,2,2,11,3,2,1,NaN,10,0,0,N
1,1,1,1,5,2,0,0,2,1,16,2,2,2,NaN,11,0,0,N
2,1,1,1,1,4,0,0,3,1,14,3,1,1,NaN,32,0,0,N
3,1,1,1,4,3,0,0,3,1,11,1,1,1,NaN,23,0,0,N
4,2,2,2,2,2,0,0,2,2,11,2,2,1,NaN,10,0,0,N


In [36]:
# Dropped the NaN in the SPEEDING label

In [37]:
df.dropna(subset=['SPEEDING'], axis=0, inplace=True)

In [38]:
df.head()

,SEVERITYCODE,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
24,2,2,2,3,3,0,0,2,2,14,1,1,1,1,14,0,0,N
43,2,1,2,4,1,0,0,1,1,28,2,2,2,1,50,0,0,N
62,1,1,1,1,4,0,0,4,3,14,1,1,1,1,32,0,0,N
123,1,1,1,3,2,0,0,2,1,14,2,2,1,1,14,0,0,N
124,2,2,2,2,2,0,0,2,2,14,1,1,1,1,10,0,0,N


In [39]:
missing_data = df.isnull()
missing_data.head(5)

,SEVERITYCODE,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
24,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
43,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
62,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
123,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
124,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [40]:
# Data set looks more balanced

In [41]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

SEVERITYCODE
False    9333
Name: SEVERITYCODE, dtype: int64

ADDRTYPE
False    9279
True       54
Name: ADDRTYPE, dtype: int64

SEVERITYDESC
False    9333
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    9332
True        1
Name: COLLISIONTYPE, dtype: int64

PERSONCOUNT
False    9333
Name: PERSONCOUNT, dtype: int64

PEDCOUNT
False    9333
Name: PEDCOUNT, dtype: int64

PEDCYLCOUNT
False    9333
Name: PEDCYLCOUNT, dtype: int64

VEHCOUNT
False    9333
Name: VEHCOUNT, dtype: int64

JUNCTIONTYPE
False    9223
True      110
Name: JUNCTIONTYPE, dtype: int64

SDOT_COLCODE
False    9333
Name: SDOT_COLCODE, dtype: int64

WEATHER
False    9325
True        8
Name: WEATHER, dtype: int64

ROADCOND
False    9330
True        3
Name: ROADCOND, dtype: int64

LIGHTCOND
False    9328
True        5
Name: LIGHTCOND, dtype: int64

SPEEDING
False    9333
Name: SPEEDING, dtype: int64

ST_COLCODE
False    9332
True        1
Name: ST_COLCODE, dtype: int64

SEGLANEKEY
False    9333
Name: SEGLANEKEY, dtype: